<a href="https://colab.research.google.com/github/ATOMScience-org/AMPL/blob/master/atomsci/ddm/examples/tutorials/09_AMPL_EDA_Part2_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA Part two: visualizing results of hyperparameter search and predictions from best models (read-only)

**Please note that this template read-only notebook is designed for begineers and provides tips on how to identify good performing models. Our suggestion is use this notebook after you have carried out your HPO search and have access to several models. Please run this notebook on systems with large diskspace and dedicated resources (RAM, CPU)**

# Scope of the tutorial:

This notebook runs through the same setup as EDA Part 2 (08_AMPL_EDA_Part2.ipynb
) but examines classification models instead of regression models. There are some new visualizations for your use.

In [ ]:
!date

Mon Aug  2 19:16:06 UTC 2021


# Install AMPL packages

In [ ]:
! pip install rdkit-pypi
! pip install deepchem

import deepchem
# print(deepchem.__version__)
! pip install umap
! pip install -U --ignore-installed numba
! pip install umap-learn
! pip install molvs
! pip install bravado

In [ ]:
import deepchem as dc

# get the Install AMPL_GPU_test.sh

!wget 'https://raw.githubusercontent.com/ATOMScience-org/AMPL/master/atomsci/ddm/examples/tutorials/config/install_AMPL_GPU_test.sh' >& /dev/null

# run the script to install AMPL
! chmod u+x install_AMPL_GPU_test.sh
! ./install_AMPL_GPU_test.sh

In [ ]:
! wget https://raw.githubusercontent.com/ATOMScience-org/AMPL/master/atomsci/ddm/examples/tutorials/datasets/H1_std.csv >& /dev/null

# Load packages and dataset

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_context("poster")
sns.set_style("whitegrid")
sns.set_palette("Set2")
pal = sns.color_palette()

import pandas as pd
import os, json, sys, glob, pickle

from atomsci.ddm.pipeline import model_pipeline as mp
from atomsci.ddm.pipeline import parameter_parser as parse
from atomsci.ddm.pipeline import perf_data
from atomsci.ddm.pipeline import compare_models as cmp

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Please use the following notebook, https://github.com/ATOMScience-org/AMPL/blob/master/atomsci/ddm/examples/tutorials/AMPL_FNL_Workshop_06052021.ipynb, to prepare `HTR3A_curated.csv` file

In [ ]:
import io
import requests

h1 = pd.read_csv("HTR3A_curated.csv", index_col=0)

In [ ]:
h1=h1[~h1.VALUE_NUM_mean.isna()]
h1=h1[h1.VALUE_NUM_mean>2]

In [ ]:
print(len(h1), h1.active.sum())
h1.hist('VALUE_NUM_mean');

# Edit some AMPL functions to include xgboost parameters
Just run this code once

In [ ]:
nan = np.float32('nan')
def get_summary_perf_tables(collection_names=None, filter_dict={}, result_dir=None, prediction_type='regression', verbose=False):
    """
    Load model parameters and performance metrics from model tracker for all models saved in the model tracker DB under
    the given collection names (or result directory if Model tracker is not available) with the given prediction type. Tabulate the parameters and metrics including:
        dataset (assay name, target, parameter, key, bucket)
        dataset size (train/valid/test/total)
        number of training folds
        model type (NN or RF)
        featurizer
        transformation type
        metrics: r2_score, mae_score and rms_score for regression, or ROC AUC for classification

    result_dir: use result_dir when the model tracker is not available. Use a list format if you have multiple result direcotries.
    """
    collection_list = []
    model_uuid_list = []
    time_built_list = []
    prediction_type_list = []
    model_type_list = []
    dataset_key_list = []
    bucket_list = []
    param_list = []
    featurizer_list = []
    desc_type_list = []
    transform_list = []
    dset_size_list = []
    splitter_list = []
    split_strategy_list = []
    split_uuid_list = []
    rf_estimators_list = []
    rf_max_features_list = []
    rf_max_depth_list = []
    best_epoch_list = []
    max_epochs_list = []
    learning_rate_list = []
    layer_sizes_list = []
    dropouts_list = []
    xgb_gamma_list = []
    xgb_learning_rate_list = []
    umap_dim_list = []
    umap_targ_wt_list = []
    umap_neighbors_list = []
    umap_min_dist_list = []
    split_uuid_list=[]


    if prediction_type == 'regression':
        score_types = ['r2_score', 'mae_score', 'rms_score']
    else:
        # TODO: add more classification metrics later
        score_types = ['roc_auc_score', 'prc_auc_score', 'accuracy_score', 'precision', 'recall_score', 'npv', 'matthews_cc', 'kappa','cross_entropy']

    subsets = ['train', 'valid', 'test']
    score_dict = {}
    ncmpd_dict = {}
    for subset in subsets:
        score_dict[subset] = {}
        for score_type in score_types:
            score_dict[subset][score_type] = []
        ncmpd_dict[subset] = []

    metadata_list_dict = {}
    if result_dir:
        if isinstance(result_dir, str):
            result_dir = [result_dir]
        for rd in result_dir:
            if rd not in metadata_list_dict:
                metadata_list_dict[rd] = []
            for dirpath, dirnames, filenames in os.walk(rd):
                if "model_metadata.json" in filenames:
                    with open(os.path.join(dirpath, 'model_metadata.json')) as f:
                        metadata_dict = json.load(f)
                    metadata_list_dict[rd].append(metadata_dict)

    for ss in metadata_list_dict:
        for i, metadata_dict in enumerate(metadata_list_dict[ss]):
            if (i % 10 == 0) and verbose:
                print('Processing collection %s model %d' % (ss, i))
            # Check that model has metrics before we go on
            if not 'training_metrics' in metadata_dict:
                continue
            collection_list.append(ss)
            model_uuid = metadata_dict['model_uuid']
            model_uuid_list.append(model_uuid)
            time_built = metadata_dict['time_built']
            time_built_list.append(time_built)

            model_params = metadata_dict['model_parameters']
            model_type = model_params['model_type']
            model_type_list.append(model_type)
            prediction_type_list.append(model_params['prediction_type'])
            featurizer = model_params['featurizer']
            featurizer_list.append(featurizer)
            if 'descriptor_specific' in metadata_dict:
                desc_type = metadata_dict['descriptor_specific']['descriptor_type']
            elif featurizer in ['graphconv', 'ecfp']:
                desc_type = featurizer
            else:
                desc_type = ''
            desc_type_list.append(desc_type)
            dataset_key = metadata_dict['training_dataset']['dataset_key']
            bucket = metadata_dict['training_dataset']['bucket']
            dataset_key_list.append(dataset_key)
            bucket_list.append(bucket)
            dset_metadata = metadata_dict['training_dataset']['dataset_metadata']
            param = metadata_dict['training_dataset']['response_cols'][0]
            param_list.append(param)
            transform_type = metadata_dict['training_dataset']['feature_transform_type']
            transform_list.append(transform_type)
            split_params = metadata_dict['splitting_parameters']
            splitter_list.append(split_params['splitter'])
            split_uuid_list.append(split_params.get('split_uuid', ''))
            split_strategy = split_params['split_strategy']
            split_strategy_list.append(split_strategy)

            if 'umap_specific' in metadata_dict:
                umap_params = metadata_dict['umap_specific']
                umap_dim_list.append(umap_params['umap_dim'])
                umap_targ_wt_list.append(umap_params['umap_targ_wt'])
                umap_neighbors_list.append(umap_params['umap_neighbors'])
                umap_min_dist_list.append(umap_params['umap_min_dist'])
            else:
                umap_dim_list.append(nan)
                umap_targ_wt_list.append(nan)
                umap_neighbors_list.append(nan)
                umap_min_dist_list.append(nan)

            if model_type == 'NN':
                nn_params = metadata_dict['nn_specific']
                max_epochs_list.append(nn_params['max_epochs'])
                best_epoch_list.append(nn_params['best_epoch'])
                learning_rate_list.append(nn_params['learning_rate'])
                layer_sizes_list.append(','.join(['%d' % s for s in nn_params['layer_sizes']]))
                dropouts_list.append(','.join(['%.2f' % d for d in nn_params['dropouts']]))
                rf_estimators_list.append(nan)
                rf_max_features_list.append(nan)
                rf_max_depth_list.append(nan)
                xgb_gamma_list.append(nan)
                xgb_learning_rate_list.append(nan)
            elif model_type == 'RF':
                rf_params = metadata_dict['rf_specific']
                rf_estimators_list.append(rf_params['rf_estimators'])
                rf_max_features_list.append(rf_params['rf_max_features'])
                rf_max_depth_list.append(rf_params['rf_max_depth'])
                max_epochs_list.append(nan)
                best_epoch_list.append(nan)
                learning_rate_list.append(nan)
                layer_sizes_list.append(nan)
                dropouts_list.append(nan)
                xgb_gamma_list.append(nan)
                xgb_learning_rate_list.append(nan)
            elif model_type == 'xgboost':
                # TODO: Add xgboost parameters
                xg_params = metadata_dict['xgb_specific']
                xgb_gamma_list.append(xg_params['xgb_gamma'])
                xgb_learning_rate_list.append(xg_params['xgb_learning_rate'])
                max_epochs_list.append(nan)
                best_epoch_list.append(nan)
                learning_rate_list.append(nan)
                layer_sizes_list.append(nan)
                dropouts_list.append(nan)
                rf_estimators_list.append(nan)
                rf_max_features_list.append(nan)
                rf_max_depth_list.append(nan)
            else:
                raise Exception('Unexpected model type %s' % model_type)

            # Get model metrics for this model
            metrics_dicts = metadata_dict['training_metrics']
            #print("Got %d metrics dicts for model %s" % (len(metrics_dicts), model_uuid))
            subset_metrics = {}
            for metrics_dict in metrics_dicts:
                if metrics_dict['label'] == 'best':
                    subset = metrics_dict['subset']
                    subset_metrics[subset] = metrics_dict['prediction_results']
            if split_strategy == 'k_fold_cv':
                dset_size = subset_metrics['train']['num_compounds'] + subset_metrics['test']['num_compounds']
            else:
                dset_size = subset_metrics['train']['num_compounds'] + subset_metrics['valid']['num_compounds'] + subset_metrics['test']['num_compounds']
            for subset in subsets:
                subset_size = subset_metrics[subset]['num_compounds']
                for score_type in score_types:
                    try:
                        score = subset_metrics[subset][score_type]
                    except KeyError:
                        score = float('nan')
                    score_dict[subset][score_type].append(score)
                ncmpd_dict[subset].append(subset_size)
            dset_size_list.append(dset_size)

    col_dict = dict(
                    collection=collection_list,
                    model_uuid=model_uuid_list,
                    time_built=time_built_list,
                    prediction_type=prediction_type_list,
                    model_type=model_type_list,
                    featurizer=featurizer_list,
                    features=desc_type_list,
                    transformer=transform_list,
                    splitter=splitter_list,
                    split_strategy=split_strategy_list,
                    split_uuid=split_uuid_list,
                    umap_dim=umap_dim_list,
                    umap_targ_wt=umap_targ_wt_list,
                    umap_neighbors=umap_neighbors_list,
                    umap_min_dist=umap_min_dist_list,
                    layer_sizes=layer_sizes_list,
                    dropouts=dropouts_list,
                    learning_rate=learning_rate_list,
                    best_epoch=best_epoch_list,
                    max_epochs=max_epochs_list,
                    rf_estimators=rf_estimators_list,
                    rf_max_features=rf_max_features_list,
                    rf_max_depth=rf_max_depth_list,
                    xgb_gamma = xgb_gamma_list,
                    xgb_learning_rate = xgb_learning_rate_list,
                    dataset_bucket=bucket_list,
                    dataset_key=dataset_key_list,
                    dataset_size=dset_size_list,
                    parameter=param_list
                    )

    perf_df = pd.DataFrame(col_dict)
    for subset in subsets:
        ncmpds_col = '%s_size' % subset
        perf_df[ncmpds_col] = ncmpd_dict[subset]
        for score_type in score_types:
            metric_col = '%s_%s' % (subset, score_type)
            perf_df[metric_col] = score_dict[subset][score_type]

    return perf_df

from atomsci.ddm.pipeline import predict_from_model as pfm

def predict_from_model_file(model_path, input_df, id_col='compound_id', smiles_col='rdkit_smiles',
                     response_col=None, is_featurized=False, dont_standardize=False):
    """
    Loads a pretrained model from a model tarball file and runs predictions on compounds in an input
    data frame.

    Args:
        model_path (str): File path of the model tarball file.

        input_df (DataFrame): Input data to run predictions on; must at minimum contain SMILES strings.

        id_col (str): Name of the column containing compound IDs. If none is provided, sequential IDs will be
        generated.

        smiles_col (str): Name of the column containing SMILES strings; required.

        response_col (str): Name of an optional column containing actual response values; if it is provided, 
        the actual values will be included in the returned data frame to make it easier for you to assess performance.

        dont_standardize (bool): By default, SMILES strings are salt-stripped and standardized using RDKit; 
        if you have already done this, or don't want them to be standardized, set dont_standardize to True.

    Return: 
        A data frame with compound IDs, SMILES strings and predicted response values. Actual response values
        will be included if response_col is provided. Standard prediction error estimates will be included
        if the model was trained with uncertainty=True. Note that the predicted and actual response
        columns will be labeled according to the response_col setting in the original training data,
        not the response_col passed to this function; e.g. if the original model response_col was 'pIC50',
        the returned data frame will contain columns 'pIC50_actual', 'pIC50_pred' and 'pIC50_std'.
    """

    input_df, pred_params = pfm._prepare_input_data(input_df, id_col, smiles_col, response_col, dont_standardize)

    has_responses = ('response_cols' in pred_params)
    pred_params = parse.wrapper(pred_params)

    pipe = mp.create_prediction_pipeline_from_file(pred_params, reload_dir=None, model_path=model_path)
    if pipe.params.model_type == 'xgboost':
        pipe.params.uncertainty = False
    pred_df = pipe.predict_full_dataset(input_df, contains_responses=has_responses, is_featurized=is_featurized,
                                        dset_params=pred_params)
    pred_df = pred_df.sort_values(by=id_col)
    return pred_df

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues, ax=None):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')
#     print(cm)
    if ax is None:
        im = plt.imshow(cm, interpolation='nearest', cmap=cmap)
#         plt.colorbar(cmap=cmap, shrink=0.7)
        tick_marks = range(0,len(classes))
        plt.xticks(tick_marks, classes, rotation=0)
        plt.yticks(tick_marks, classes, rotation=90, va='center')

        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
    else:
        im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
#         plt.colorbar(im, shrink=0.7)
        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            ax.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
#         ax.tight_layout()
        ax.set_ylabel('True label')
        ax.set_xlabel('Predicted label')
        tick_marks = range(0,len(classes))
        ax.set_xticks(tick_marks)
        ax.set_xticklabels(classes, rotation=0)
        ax.set_yticks(tick_marks)
        ax.set_yticklabels(classes, rotation=90, va='center')
    return im

# Visualize hyper parameter search
- you can use regression or classification models for this, just choose a different metric for the y-axis
- box plots work well if you did a grid search, scatter plots better if you did a HPO optimization
- LOTS of parameters to look at, think about questions you have for the data:
  - how do train, valid and test metrics look?
  - which features are best for modeling the data?
  - which hp's are the best?
  - which models are the best?

#### Get a table of performance metrics of all models in your directory

In [ ]:
# slow the first time after connecting your google drive
result_dir = '/content/drive/MyDrive/Columbia_E4511/HTR3A_models'
perf_df = get_summary_perf_tables(collection_names=None, filter_dict={}, prediction_type = 'classification', result_dir=result_dir, verbose=False)
perf_df = perf_df[perf_df.rf_estimators!=500]
perf_df = perf_df[perf_df.prediction_type=='classification']
perf_df.sort_values(by="valid_roc_auc_score", ascending=False).head()

In [ ]:
perf_df.groupby(by=['model_type', 'features']).count()[['model_uuid']]

#### Examine ROC_AUC scores from train, valid and test sets
- examine other metrics instead. Do you see differences?
- Play around with hues from different hyperparameters. Do you see any trends?

In [ ]:
# what are the scores like for train, valid and test sets?

scoretype='roc_auc_score'
subset='valid'
winnertype= f'{subset}_{scoretype}'
plot_df=perf_df[[f"train_{scoretype}",f"valid_{scoretype}",f"test_{scoretype}"]]
# turn off sorting if you have a ton of models.. slow
plot_df=plot_df.sort_values(f"valid_{scoretype}")

fig, ax = plt.subplots(1,4,figsize=(40,10))
sns.kdeplot(perf_df[f'train_{scoretype}'], label="train",ax=ax[0])
sns.kdeplot(perf_df[f'valid_{scoretype}'], label="valid",ax=ax[0])
sns.kdeplot(perf_df[f'test_{scoretype}'], label="test",ax=ax[0])
ax[0].set_xlabel(f'{scoretype}')

ax[0].legend(loc="upper left")
plot_df = perf_df[perf_df.model_type=="RF"]
huefeat = 'rf_estimators'
plot_df=plot_df[[f"train_{scoretype}",f"valid_{scoretype}",f"test_{scoretype}", huefeat, 'model_uuid']]
plot_df=plot_df.sort_values(f"valid_{scoretype}")
plot_df=plot_df.melt(id_vars=['model_uuid',huefeat])
sns.lineplot(data=plot_df, x='variable', y='value', units ='model_uuid', estimator=None, hue=huefeat, legend='brief', ax = ax[1]);
ax[1].set_ylim(perf_df[f'test_{scoretype}'].min()-.1,1)
ax[1].set_title('RF')

plot_df = perf_df[perf_df.model_type=="NN"]
huefeat = 'learning_rate'
plot_df=plot_df[[f"train_{scoretype}",f"valid_{scoretype}",f"test_{scoretype}", huefeat, 'model_uuid']]
plot_df=plot_df.sort_values(f"valid_{scoretype}")
plot_df=plot_df.melt(id_vars=['model_uuid',huefeat])
sns.lineplot(data=plot_df, x='variable', y='value', units ='model_uuid', estimator=None, hue=huefeat, legend='brief', ax = ax[2]);
ax[2].set_ylim(perf_df[f'test_{scoretype}'].min()-.1,1)
ax[2].set_title('NN')

plot_df = perf_df[perf_df.model_type=="xgboost"]
huefeat = 'xgb_learning_rate'
plot_df=plot_df[[f"train_{scoretype}",f"valid_{scoretype}",f"test_{scoretype}", huefeat, 'model_uuid']]
plot_df=plot_df.sort_values(f"valid_{scoretype}")
plot_df=plot_df.melt(id_vars=['model_uuid',huefeat])
sns.lineplot(data=plot_df, x='variable', y='value', units ='model_uuid', estimator=None, hue=huefeat, legend='brief', ax = ax[3]);
ax[3].set_ylim(perf_df[f'test_{scoretype}'].min()-.1,1)
ax[3].set_title('xgboost')

fig.suptitle(f"{scoretype}s for HTR3A regression models");

#### Examine ROC_AUC scores for each feature set
- what do you think of the variability among scores for different model types? Is it different for MAE or RMS scores?

In [ ]:
# which feature set works best for each model type?

fig,ax = plt.subplots(1,3,sharey=True, figsize=(26,8))
plot_df = perf_df[perf_df.model_type=='RF']
sns.boxplot(data=plot_df, x="features", y=f"valid_{scoretype}", width = 0.8, ax=ax[0]);
# plot_df = perf_df[perf_df.model_type=='NN']
# sns.boxplot(data=plot_df, x="features", y=f"valid_{scoretype}", width = 0.8, ax=ax[1]);
# plot_df = perf_df[perf_df.model_type=='xgboost']
# sns.boxplot(data=plot_df, x="features", y=f"valid_{scoretype}", width = 0.8, ax=ax[2]);
ax[0].set_title("RF");
ax[1].set_title("NN");
ax[2].set_title("XG");
plt.tight_layout()

#### Examine the effect of RF hyperparameters on model performance
- any major differences between feature sets?
- did you capture the right range of HPs or do you need to expand / zero in on a particular range?

In [ ]:
modtype = 'RF'
feat1 = 'rf_estimators'
feat2 = 'rf_max_depth'
feat3 = 'rf_max_features'
sub_df = perf_df[perf_df.model_type==modtype]

fig, ax = plt.subplots(3,3,sharey=True, figsize=(26,18))
plot_df=sub_df[sub_df.features=='ecfp']
sns.boxplot(data=plot_df, x=feat1, y=f"valid_{scoretype}", ax=ax[0,0])
sns.boxplot(data=plot_df, x=feat2, y=f"valid_{scoretype}", ax=ax[0,1])
ax[0,1].set_title(f"{modtype} - {plot_df.features.iloc[0]}")
sns.boxplot(data=plot_df, x=feat3, y=f"valid_{scoretype}", ax=ax[0,2])

plot_df=sub_df[sub_df.features=='mordred_filtered']
sns.boxplot(data=plot_df, x=feat1, y=f"valid_{scoretype}", ax=ax[1,0])
sns.boxplot(data=plot_df, x=feat2, y=f"valid_{scoretype}", ax=ax[1,1])
ax[1,1].set_title(f"{modtype} - {plot_df.features.iloc[0]}")
sns.boxplot(data=plot_df, x=feat3, y=f"valid_{scoretype}", ax=ax[1,2])

plot_df=sub_df[sub_df.features=='rdkit_raw']
sns.boxplot(data=plot_df, x=feat1, y=f"valid_{scoretype}", ax=ax[2,0])
sns.boxplot(data=plot_df, x=feat2, y=f"valid_{scoretype}", ax=ax[2,1])
ax[2,1].set_title(f"{modtype} - {plot_df.features.iloc[0]}")
sns.boxplot(data=plot_df, x=feat3, y=f"valid_{scoretype}", ax=ax[2,2])
fig.tight_layout();

#### Examine NN hyperparameters
- are training or test sets different?

In [ ]:
# i didn't train any NN classification models

# modtype = 'NN'
# feat1 = 'dropouts'
# feat2 = 'learning_rate'
# feat3 = 'layer_sizes'
# sub_df = perf_df[perf_df.model_type==modtype]

# fig, ax = plt.subplots(3,3,sharey=True,figsize=(26,18))
# plot_df=sub_df[sub_df.features=='ecfp']
# sns.boxplot(data=plot_df, x=feat1, y=f"valid_{scoretype}", ax=ax[0,0])
# sns.boxplot(data=plot_df, x=feat2, y=f"valid_{scoretype}", ax=ax[0,1])
# ax[0,1].set_title(f"{modtype} - {plot_df.features.iloc[0]}")
# sns.boxplot(data=plot_df, x=feat3, y=f"valid_{scoretype}", ax=ax[0,2])

# plot_df=sub_df[sub_df.features=='mordred_filtered']
# sns.boxplot(data=plot_df, x=feat1, y=f"valid_{scoretype}", ax=ax[1,0])
# sns.boxplot(data=plot_df, x=feat2, y=f"valid_{scoretype}", ax=ax[1,1])
# ax[1,1].set_title(f"{modtype} - {plot_df.features.iloc[0]}")
# sns.boxplot(data=plot_df, x=feat3, y=f"valid_{scoretype}", ax=ax[1,2])

# plot_df=sub_df[sub_df.features=='rdkit_raw']
# sns.boxplot(data=plot_df, x=feat1, y=f"valid_{scoretype}", ax=ax[2,0])
# sns.boxplot(data=plot_df, x=feat2, y=f"valid_{scoretype}", ax=ax[2,1]); ax[2,1].tick_params(axis='x', rotation=45);
# ax[2,1].set_title(f"{modtype} - {plot_df.features.iloc[0]}")
# sns.boxplot(data=plot_df, x=feat3, y=f"valid_{scoretype}", ax=ax[2,2])
# fig.tight_layout();

#### Examine XGBoost hyperparameters
- what is the third graph?
- how do scores compare when plotted against each other?


In [ ]:
# i didn't train xgboost classification models

# modtype = 'xgboost'
# feat1 = 'xgb_gamma'
# feat2 = 'xgb_learning_rate'
# feat3 = 'valid_mae_score'
# sub_df = perf_df[perf_df.model_type==modtype]

# fig, ax = plt.subplots(3,3,sharey=True, figsize=(26,18))
# plot_df=sub_df[sub_df.features=='ecfp']
# sns.boxplot(data=plot_df, x=feat1, y=f"valid_{scoretype}", ax=ax[0,0])
# sns.boxplot(data=plot_df, x=feat2, y=f"valid_{scoretype}", ax=ax[0,1])
# ax[0,1].set_title(f"{modtype} - {plot_df.features.iloc[0]}")
# sns.scatterplot(data=plot_df, x=feat3, y=f"valid_{scoretype}", ax=ax[0,2])

# plot_df=sub_df[sub_df.features=='mordred_filtered']
# sns.boxplot(data=plot_df, x=feat1, y=f"valid_{scoretype}", ax=ax[1,0])
# sns.boxplot(data=plot_df, x=feat2, y=f"valid_{scoretype}", ax=ax[1,1])
# ax[1,1].set_title(f"{modtype} - {plot_df.features.iloc[0]}")
# sns.scatterplot(data=plot_df, x=feat3, y=f"valid_{scoretype}", ax=ax[1,2])

# plot_df=sub_df[sub_df.features=='rdkit_raw']
# sns.boxplot(data=plot_df, x=feat1, y=f"valid_{scoretype}", ax=ax[2,0])
# sns.boxplot(data=plot_df, x=feat2, y=f"valid_{scoretype}", ax=ax[2,1]); ax[2,1].tick_params(axis='x', rotation=45);
# ax[2,1].set_title(f"{modtype} - {plot_df.features.iloc[0]}")
# sns.scatterplot(data=plot_df, x=feat3, y=f"valid_{scoretype}", ax=ax[2,2])
# fig.tight_layout();

# Choose best model and examine predictions (classification models)
Metrics to assess classification models: (google definitions)
- cross_entropy
- kappa
- matthews_cc
- npv
- prc_auc_score
- precision
- recall_score
- roc_auc_score
- balanced_accuracy

### Challenge: try adding balanced accuracy scores into this plot

#### select best models & visualize metrics with radar plot

In [ ]:
# best models per metric
top_xnt_model = perf_df[perf_df.valid_cross_entropy==perf_df.valid_cross_entropy.min()] # minimize cross entropy, maximize rest
top_kap_model = perf_df[perf_df.valid_kappa==perf_df.valid_kappa.max()]
top_mcc_model = perf_df[perf_df.valid_matthews_cc==perf_df.valid_matthews_cc.max()]
top_npv_model = perf_df[perf_df.valid_npv==perf_df.valid_npv.max()]
top_prc_model = perf_df[perf_df.valid_prc_auc_score==perf_df.valid_prc_auc_score.max()]
top_pre_model = perf_df[perf_df.valid_precision==perf_df.valid_precision.max()]
top_rec_model = perf_df[perf_df.valid_recall_score==perf_df.valid_recall_score.max()]
top_roc_model = perf_df[perf_df.valid_roc_auc_score==perf_df.valid_roc_auc_score.max()] 

topmods = pd.concat([top_xnt_model, top_kap_model, top_mcc_model, top_npv_model, top_prc_model, top_pre_model, top_rec_model, top_roc_model])
# note: fewer different model_uuids here than metrics from some models winning at more than one metric
topmods = topmods.drop_duplicates()
topmods

In [ ]:
import math
plot_df=topmods
categories = ['valid_roc_auc_score','valid_prc_auc_score','valid_accuracy_score','valid_precision', 'valid_recall_score', 'valid_npv', 'valid_matthews_cc', 'valid_kappa', 'valid_cross_entropy']
# categories_2 = ['best_train_r2_score','best_valid_r2_score','best_train_mae_score','best_valid_mae_score','best_train_rms_score',  'best_valid_rms_score']
N=len(categories)
angles = [n / float(N) * 2 * math.pi for n in range(N)]
angles += angles[:1]
fig, ax = plt.subplots(1,1,figsize=(10,10), subplot_kw=dict(polar=True))
ax.set_theta_offset(math.pi / 2)
ax.set_theta_direction(-1)
ax.set_rlabel_position(0)
ax.set_yticks([.25,0.5,0.75,1])
ax.set_ylim(0,1)
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
for ilocn in plot_df.index:
    values=plot_df.loc[ilocn][categories].values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=2, linestyle='solid', label=f"Model # {ilocn}")
#         ax.fill(angles, values, 'b', alpha=0.05)
ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
ax.set_title(f"Top model metrics", y=1.06);
fig.tight_layout()

#### select models and generate predictions
- predict on the same dataset as the model was trained on first

In [ ]:
from atomsci.ddm.pipeline import predict_from_model as pfm

In [ ]:
preds = []
for i, mod in topmods.iterrows():
  model_path = f"{mod.collection}/{mod.dataset_key.split('/')[-1].strip('.csv')}_model_{mod.model_uuid}.tar.gz"
  split_df = pd.read_csv(f"/content/drive/MyDrive/Columbia_E4511/HTR3A_curated_train_valid_test_scaffold_{mod.split_uuid}.csv")
  if mod.features in ['rdkit_raw', 'mordred_filtered']:
    feat_df = pd.read_csv(f'/content/drive/MyDrive/Columbia_E4511/scaled_descriptors/HTR3A_curated_with_{mod.features}_descriptors.csv')
    feat_df = feat_df[~feat_df.VALUE_NUM_mean.isna()]
    feat_df = feat_df[feat_df.VALUE_NUM_mean>2]
    is_featurized = True
  else:
    feat_df = pd.read_csv(mod.dataset_key)
    feat_df = feat_df[~feat_df.VALUE_NUM_mean.isna()]
    feat_df = feat_df[feat_df.VALUE_NUM_mean>2]
    is_featurized = False
  pred_df = pfm.predict_from_model_file(model_path= model_path, input_df=feat_df, id_col='compound_id', smiles_col = 'base_rdkit_smiles', 
                        response_col = 'active', is_featurized = is_featurized, dont_standardize=True)
  pred_df = pd.merge(pred_df, split_df, left_on='compound_id', right_on='cmpd_id')
  pred_df = pred_df.rename(columns={'active_actual':'active'})
  preds.append(pred_df)

In [ ]:
preds[0]

#### Plot confusion matrices and other plots
- confusion matrix (active vs inactive)
- reciever-operating curve
  - what does a "good" curve look like?
- precision-recall curve
  - what does a "good" curve look like?
- when is it appropriate to use predicted class versus class probabilities?
- what do the graphs look like if you change to _pred from _prob?

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score, r2_score, precision_recall_curve, average_precision_score, confusion_matrix, precision_score, recall_score, balanced_accuracy_score
import itertools
sns.set_style('white')

In [ ]:
full_df=preds[0]
model_uuid = topmods.model_uuid.tolist()[0]
response_col = 'active'
# graph
fig, ax = plt.subplots(2,2, sharex=False, sharey=False, figsize=(15,15))
ax=ax.ravel()

# training set    
pred_df=full_df[full_df.subset=='train']
# roc curve
roc_auc = roc_auc_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
fpr, tpr, _ = roc_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
lw = 2
ax[0].plot(fpr, tpr, lw=lw, label='Train ROC AUC = %0.2f' % roc_auc, color = pal[3])
# prc 
average_precision = average_precision_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
precision, recall, _ = precision_recall_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
line_kwargs = {"drawstyle": "steps-post"}
line_kwargs["label"] = (f"Train AP = "
                        f"{average_precision:0.2f}")
ax[1].plot(recall, precision, color=pal[3], **line_kwargs) 

# validation set
pred_df=full_df[full_df.subset=='valid']
# ROC AUC curve
roc_auc = roc_auc_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
fpr, tpr, _ = roc_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
lw = 2
line_kwargs = {"drawstyle": "steps-post"}
ax[0].plot(fpr, tpr, lw=lw, label='Valid ROC AUC = %0.2f' % roc_auc, color = pal[4], **line_kwargs)
# PR curve
average_precision = average_precision_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
precision, recall, _ = precision_recall_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
line_kwargs = {"drawstyle": "steps-post"}
line_kwargs["label"] = (f"Valid AP = "
                        f"{average_precision:0.2f}")
ax[1].plot(recall, precision, color=pal[4], **line_kwargs)
# confusion matrix - valid set
cm=confusion_matrix(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
class_names = ['inactive','active']
im = plot_confusion_matrix(cm, classes=class_names, normalize=False, cmap=sns.cubehelix_palette(rot=0, start=2.40, as_cmap=True), ax=ax[2])
fig.colorbar(mappable=im, ax=ax[2], shrink=0.7)
ax[2].set_title("Valid CM")

# test set    
pred_df=full_df[full_df.subset=='test']
# ROC AUC curve
roc_auc = roc_auc_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
fpr, tpr, _ = roc_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
lw = 2
line_kwargs = {"drawstyle": "steps-post"}
ax[0].plot(fpr, tpr, lw=lw, label=' Test ROC AUC = %0.2f' % roc_auc, color = pal[5], **line_kwargs)
ax[0].plot([0, 1], [0, 1], lw=lw, linestyle='--')
ax[0].set_xlim([-0.05, 1.05])
ax[0].set_ylim([-0.05, 1.05])
ax[0].set_xlabel('False Positive Rate')
ax[0].set_ylabel('True Positive Rate')
ax[0].legend(loc="lower right")
# PR curve
average_precision = average_precision_score(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
precision, recall, _ = precision_recall_curve(pred_df[f'{response_col}'], pred_df[f'{response_col}_prob'])
line_kwargs = {"drawstyle": "steps-post"}
line_kwargs["label"] = (f" Test AP = "
                        f"{average_precision:0.2f}")
ax[1].plot(recall, precision, color=pal[5], **line_kwargs)
pos_label=1
info_pos_label = (f" (Positive label: {pos_label})")
xlabel = "Recall" + info_pos_label
ylabel = "Precision" + info_pos_label
ax[1].set(xlabel=xlabel, ylabel=ylabel)
ax[1].set_xlim([-0.05, 1.05])
ax[1].set_ylim([-0.05, 1.05])
ax[1].legend(loc="lower right")
# confusion matrix
cm=confusion_matrix(pred_df[f'{response_col}'], pred_df[f'{response_col}_pred'])
im=plot_confusion_matrix(cm, classes=class_names, normalize=False, cmap=sns.cubehelix_palette(rot=0, start=2.40, as_cmap=True), ax=ax[3])
fig.colorbar(mappable=im, ax=ax[3], shrink=0.7)
ax[3].set_title("Test CM")
plt.tight_layout()
fig.suptitle(f'{model_uuid}', y=1.01);

# Predict on new data (classification)

#### Generate predictions on a list of compounds from another target (such as provided by your partners)
- there will be activity values associated with this target. You can explore overlaps between the two targets
- here, my best models are predicting activity of the serotonin receptor, HTR3A
- i will compare with data from the serotonin transporter, SLC6A4

In [ ]:
from atomsci.ddm.utils import curate_data
from atomsci.ddm.utils import struct_utils as su

In [ ]:
# new data must have curated smiles strings, with rdkit standardization, featurizations, etc.
newcmpds = pd.read_csv("/content/drive/MyDrive/Columbia_E4511/SLC6A4_chembl.csv", sep=";")
newcmpds = curate_data.average_and_remove_duplicates('pChEMBL Value', 100, False, data=newcmpds, compound_id='Molecule ChEMBL ID', smiles_col='Smiles')
newcmpds['base_rdkit_smiles']=su.base_smiles_from_smiles(newcmpds.Smiles.tolist())
newcmpds = newcmpds.rename(columns = {'Molecule ChEMBL ID':'compound_id'})
newcmpds = curate_data.aggregate_assay_data(newcmpds, value_col='VALUE_NUM_mean', output_value_col='VALUE_NUM_mean', label_actives=True, active_thresh=7, id_col='compound_id', smiles_col='base_rdkit_smiles', relation_col = 'Standard Relation')
newcmpds = newcmpds[~newcmpds.base_rdkit_smiles.isna()]
newcmpds = newcmpds[~newcmpds.VALUE_NUM_mean.isna()]
newcmpds = newcmpds[newcmpds.VALUE_NUM_mean!='']
newcmpds.to_csv("/content/drive/MyDrive/Columbia_E4511/SLC6A4_chembl_cur.csv", index = False)
newcmpds.head()

In [ ]:
# next cells will be very slow if you have mordred features. 
# Get the featurized data first and then say featurized = true

In [ ]:
preds = []
# for i, mod in topmods.iloc[1,:].iterrows(): 
# run with one ecfp model for demo
mod = topmods.iloc[0,:]
model_path = f"{mod.collection}/{mod.dataset_key.split('/')[-1].strip('.csv')}_model_{mod.model_uuid}.tar.gz"
pred_df = pfm.predict_from_model_file(model_path= model_path, input_df=newcmpds, id_col='compound_id', smiles_col = 'base_rdkit_smiles', 
                      response_col = 'active', is_featurized = False, dont_standardize=True)
pred_df = pred_df.rename(columns={'active_actual':'active'})
preds.append(pred_df)

In [ ]:
# remember - active here is the IC50 values for other target SLC6A4 (that's where the dataset comes from)
# active_pred is your predictions for YOUR target (HTR3A), not the other target
# SLC6A4 is a serotonin transporter
# HTR3A is a serotonin receptor
# related targets, but not the same. Will be interesting to see overlaps in activity
preds[0]

#### Visualize predictions that don't have ground truth associated with them


In [ ]:
# active prob hist
fig, ax = plt.subplots(1, sharex=True, sharey=True, figsize = (10,10))
preds[0].hist('active_prob', bins=10, ax=ax)
ax.set_title(topmods.model_uuid.tolist()[0]);

In [ ]:
# confusion matrix - your target predictions vs other target classes
cm=confusion_matrix(preds[0][f'{response_col}'], preds[0][f'{response_col}_pred'])
fig, ax = plt.subplots(1,figsize=(10,10))
class_names = ['inactive','active']
im = plot_confusion_matrix(cm, classes=class_names, normalize=False, cmap=sns.cubehelix_palette(rot=0, start=2.40, as_cmap=True), ax=ax)
fig.colorbar(mappable=im, ax=ax, shrink=0.7)
ax.set_xlabel('Predicted - my target (HTR3A)')
ax.set_ylabel('True label - other target (SLC6A4)')
ax.set_title("2-target CM");

#### Tanimoto distance from train to new compounds
- what does a tanimoto distance of zero represent?
- what do you think about the distances between your training data and the new data? are the compounds similar or different?
- what do you think this means for the uncertainty of new predictions?

In [ ]:
from atomsci.ddm.pipeline import chem_diversity as cd

In [ ]:
htr3a=h1
calc_type='nearest'
dist_metric='tanimoto'
smiles_lst2=newcmpds.base_rdkit_smiles.tolist()
htr3a=htr3a.merge(split_df, left_on='compound_id', right_on='cmpd_id')
smiles_lst1=htr3a[htr3a.subset=='train'].base_rdkit_smiles.tolist()
dists=cd.calc_dist_smiles('ECFP',dist_metric,smiles_lst2,smiles_lst1,calc_type)
distsdf=pd.DataFrame([smiles_lst2,list(dists)], columns=range(len(smiles_lst2)), index=['smiles','dists']).T
df=newcmpds
df=df.merge(distsdf, left_on='base_rdkit_smiles', right_on='smiles')
print(len(set(smiles_lst2)-set(smiles_lst1)))
print(list(df.loc[df.dists==0, 'compound_id']))

In [ ]:
fig, axes = plt.subplots(1, figsize=(10,10))
sns.distplot(dists, ax=axes, bins=np.arange(-0.05,1,0.05))
axes.set_title("Tanimoto distance of compounds to training set of best models");

# Explore the domain of applicability of your model

#### Make sure you have featurized data to use for UMAPs
graphconv you can't do umaps

In [ ]:
newcmpds.to_csv("/content/drive/MyDrive/Columbia_E4511/SLC6A4_chembl_cur.csv")

In [ ]:
# to create umaps on training data, you need features for the model type.
# rdkit and mordred should already be featurized if you built models with them
# ecfp doesn't have saved features, so extract them now for the training data and new data
# just change the train_file to featurize the other dataset
from atomsci.ddm.pipeline import featurization as feat

train_file = "/content/drive/MyDrive/Columbia_E4511/SLC6A4_chembl_cur.csv"
response_col = "VALUE_NUM_mean"
compound_id = "compound_id"
smiles_col = "base_rdkit_smiles"
result_dir = "/content/drive/MyDrive/Columbia_E4511/HTR3A_models"
params = {
        "system": "LC",
        "lc_account": 'None',
        "datastore": "False",
        "save_results": "False",
        "data_owner": "username",
        "dataset_key": train_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "previously_split": "False",
        "split_only": "True",
        # "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": result_dir,
        "featurizer":"ecfp"
    }
pparams = parse.wrapper(params)
MP = mp.ModelPipeline(pparams)
featurization=None
# comment out this line after splitting once so you don't re-split
MP.run_mode = 'training'
MP.params.split_only = True
MP.params.previously_split = False
if featurization is None:
    featurization = feat.create_featurization(MP.params)
MP.featurization = featurization
MP.load_featurize_data()

In [ ]:
ecfp = MP.data.dataset.X
y=MP.data.dataset.y
ids=MP.data.dataset.ids
yids = [ids, y]
yids.extend(ecfp.T)
ecfpfeats = pd.DataFrame(yids).T
ecfp_colnames = ['compound_id', 'mol_wt']
col2 = list(range(0,1024))
col2 = ['ecfp_'+ str(x) for x in col2]
ecfp_colnames.extend(col2)
ecfpfeats.columns = ecfp_colnames
ecfpfeats.to_csv('/content/drive/MyDrive/Columbia_E4511/scaled_descriptors/SLC6A4_chembl_cur_with_ecfp_descriptors.csv')

#### UMAP projection of new compounds onto the training dataset
- create umap of training data
- use the mapper object to project new compounds onto the training data
- visualize prediction uncertainty or ADI as hue

In [ ]:
import umap

In [ ]:
preds[0]

In [ ]:
cols_dict={
    'ecfp':('ecfp_0','ecfp_1023'),
    'mordred_filtered':('ABC','SsBr'),
    'rdkit_raw':('MaxEStateIndex','fr_urea'),
}
# features model was created on, for training data
feat_type = 'ecfp'
feats = pd.read_csv(f"/content/drive/MyDrive/Columbia_E4511/scaled_descriptors/HTR3A_curated_with_{feat_type}_descriptors.csv")
valuecol=[f'active']
feats = feats.merge(split_df, left_on='compound_id', right_on='cmpd_id')
feats = feats.merge(h1)
feats = feats[feats.subset=='train']
feats = feats.dropna(subset = valuecol).reset_index(drop=True)
# select only the non-na feature columns to run umap on
firstcol, lastcol=cols_dict[feat_type]
maptrain = feats.loc[:,firstcol:lastcol]
maptrain = maptrain.dropna(axis='columns')
maptraincols = maptrain.columns
# get same feature columns from new compound df
featnew=pd.read_csv(f"/content/drive/MyDrive/Columbia_E4511/scaled_descriptors/SLC6A4_chembl_cur_with_{feat_type}_descriptors.csv", index_col=False)
featnew=featnew.merge(preds[0])
mapnew = featnew.loc[:,firstcol:lastcol]
mapnew = mapnew.dropna(axis='columns')
mapnewcols = mapnew.columns
allcols = list(set(maptraincols).intersection(set(mapnewcols)))
maptrain = maptrain[allcols]
mapnew= mapnew[allcols]
# create umap on training data
mapper=umap.UMAP(n_neighbors=15, n_components=2, metric='jaccard', random_state=42).fit(maptrain)
maptrain_coords = pd.DataFrame(mapper.embedding_, columns=('UMAP_X', 'UMAP_Y'))
feats=pd.concat([feats,maptrain_coords], axis=1)
# use mapper to create umap coords for new data
mapnew_coords = pd.DataFrame(mapper.transform(mapnew), columns = ('UMAP_X', 'UMAP_Y'))
featnew=pd.concat([featnew,mapnew_coords], axis=1)
featnew=featnew.merge(newcmpds)
featnew=featnew.sort_values('active')

##### How would you interpret the probability output? (active_prob)

In [ ]:
# plot
fig, ax = plt.subplots(1,2, figsize=(30,15), gridspec_kw={'width_ratios': [1, 1.2]})
# plot with active / inactive labels
sns.scatterplot(x=feats['UMAP_X'], y=feats["UMAP_Y"], s=50, hue=feats['active'].map({0:False, 1:True}), palette=['#e0e0e0','#b3b3b3'], ax=ax[0])
sns.scatterplot(x=featnew['UMAP_X'], y=featnew["UMAP_Y"], hue=featnew[f'active_pred'], palette='Set2', marker = '^', legend='full', ax=ax[0])
handles, _ = ax[0].get_legend_handles_labels()
ax[0].legend([handles[1],handles[-1]], [f'Training Hits', f'Pred Hits'], fontsize=12)
ax[0].set_title(f'Hits on {feat_type} training data UMAP (n={len(feats)})')
# plot with continuous labels
valcol=f'active_prob'
# reduce number of points to plot
plotnew = featnew[(featnew[valcol]<0.05) | (featnew[valcol]>0.8)] # highest and lowest values = more certain
norm = plt.Normalize(plotnew[valcol].min(), plotnew[valcol].max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])
sns.scatterplot(x=feats['UMAP_X'], y=feats["UMAP_Y"], s=50, hue=feats['active'], palette=['#e0e0e0','#b3b3b3'], ax=ax[1])
sns.scatterplot(x=plotnew['UMAP_X'], y=plotnew["UMAP_Y"], hue=plotnew[valcol], palette='viridis', marker = '^', ax=ax[1]);
# Remove the legend and add a colorbar
ax[1].get_legend().remove()
ax[1].figure.colorbar(sm)
ax[1].set_title(f'Compound cert. on {feat_type} training data UMAP (n={len(feats)})');
plt.tight_layout()